# 流畅的 Python

## 第二部分 数据结构

## 第2章 序列构成的数组

Python 用统一的风格处理序列数据。深入了解 Python 中的不同序列类型，不但能让我们避免重新发明轮子，它们的 API 还能帮助我们把自己定义的 API 设计得跟原生的序列类型一样，或者是跟未来可能出现的序列类型保持兼容。

Python handles sequential data in a unified style. An insight into the different sequence types in Python not only prevents us from reinventing the wheel, but their API also helps us design our own API as a native sequence type, or is compatible with the possible sequence types in the future.

### 2.1 内置序列类型概览

容器序列 list, tuple, collections.deque

扁平序列 str, bytes, bytearray, memoryview, array.array
容器序列里存放的是它们所包含的任意类型的对象的引用，而扁平序列里存放的是值而不是引用，扁平序列其实是一段连续的内存空间。

按照能够被修改来分类：
可变序列 list, bytearray, array.array, collections.deque, memoryview
不可变序列 tuple, str,bytes

### 2.2 列表推导和生成器表达式

通常原则：只用列表推导来创建新的列表，并且尽量保持简短。

句法提示：Python 会忽略代码里[],{}和()中的换行，因此如果你的代码里有多行的列表、列表推导、生成器表达式、字典这一类的，可以省略不太好看的续行符\。

对于 Python 3，列表推导不会再有变量泄漏的问题。列表推导、生成器表达式，以及同它们很相似的集合（set）推导和字典（dict）推导，在 Python 3 中都有了自己的局部作用域，就像函数似的。表达式内部的变量和赋值只在局部作用，表达式的上下文里的同名变量还可以被正常引用，局部变量并不会影响到它们。

列表推导可以帮助我们把一个序列或是其他可迭代类型中的元素过滤或是加工，然后再新建一个列表。

In [2]:
symbols = '@#$%^&*'
codes = [ord(symbol) for symbol in symbols]
codes

[64, 35, 36, 37, 94, 38, 42]

#### 列表推导同 filter 和 map 的比较

In [7]:
symbols = '￥¥£₠￡$'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
print(beyond_ascii)
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
print(beyond_ascii)

[65509, 165, 163, 8352, 65505]
[65509, 165, 163, 8352, 65505]


#### 笛卡尔积

R x S 笛卡尔积是一个列表，列表里的元素是由输入的可迭代类型的元素对构成的元组，因此笛卡尔积列表的长度等于输入变量的长度的乘积。

In [1]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
# 先以颜色排列，再以尺码排列
tshirts = [(color, size) for color in colors
                         for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [2]:
# 先以尺码排列，再以颜色排列
tshirts = [(color, size) for size in sizes
                         for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

#### 生成器表达式

In [3]:
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


In [4]:
# 用生成器表达式初始化元组
symbols = '@#$%^&*'
print(tuple(ord(symbol) for symbol in symbols))

(64, 35, 36, 37, 94, 38, 42)


In [6]:
# 用生成器表达式初始化数组
import array
print(array.array('I', (ord(symbol) for symbol in symbols)))

array('I', [64, 35, 36, 37, 94, 38, 42])


### 2.3 元组不仅仅是不可变的列表

#### 元组和记录

如果把元组当作一些字段的集合，那么数量和位置信息就变得非常重要了。如果在任何元组内对元素排序，这些元素所携带的信息就会丢失，因为这些信息是跟它们的位置有关的。

In [8]:
# 把元组用作记录
lax_cordinates = (33.9, -118.3)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

# 排序，迭代过程中 passport 变量被绑定到每个元组上
for passport in sorted(traveler_ids):
    # %s 格式运算符能被匹配到对应的元组元素上
    print('%s%s' % passport)
    
# 拆包 unpacking
for country, _ in traveler_ids:
    print(country)

BRACE342567
ESPXDA205856
USA31195855
USA
BRA
ESP


#### 元组拆包

